In this ipynb file, I m trying to incorporate some related issues which are leading of shift in centroid

In [ ]:
from movement import sample_data
from movement.io import load_bboxes

SHA256 hash of downloaded file: 40a6d79722dcaf7cee39627959e46a1bc10a6e1405842a7c2993d6d69b4c8195
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


In [2]:
dataset_dict = sample_data.fetch_dataset_paths(
    "VIA_single-crab_MOCA-crab-1.csv",
    with_video=True,  # download associated video
)

file_path = dataset_dict["bboxes"]
print(file_path)

ds = load_bboxes.from_via_tracks_file(
    file_path, use_frame_numbers_from_file=True
)

/home/mahima/.movement/data/bboxes/VIA_single-crab_MOCA-crab-1.csv


If we copy the address of the temp file ('/home/mahima/.movement/data/bboxes/VIA_single-crab_MOCA-crab-1.csv') and run it on napari, the csv file will track the centroid of the crab.

In [ ]:
# Now I will be converting this original ds x array back to a csv file
# Ideally we expect it to trace the centroid since there is
# no change in anything ,its just back and forth conversion
# but
# I had to include the centroid shifting code in the following
# code t align it at the centre

In [ ]:
import json


def convert_to_via_format(ds, filename):
    """Convert Xarray dataset to VIA CSV format with
    correctly formatted bounding boxes.
    """
    df = ds.to_dataframe().reset_index()

    # Pivot data to organize x, y, width, height correctly
    df_pivot = df.pivot(
        index=["time"], columns="space", values=["position", "shape"]
    )

    # Flatten multi-index column names
    df_pivot.columns = ["x_center", "y_center", "width", "height"]
    df_pivot = df_pivot.reset_index()

    # Convert centroid back to top-left corner
    df_pivot["x"] = df_pivot["x_center"] - df_pivot["width"] / 2
    df_pivot["y"] = df_pivot["y_center"] - df_pivot["height"] / 2

    # Generate correct filenames
    df_pivot["filename"] = df_pivot["time"].apply(
        lambda t: f"/crab_1/{t:05d}.jpg"
    )

    # Add metadata fields
    df_pivot["file_size"] = 0
    df_pivot["file_attributes"] = json.dumps({"shot": 123})
    df_pivot["region_count"] = 1
    df_pivot["region_id"] = 0
    df_pivot["region_attributes"] = json.dumps({"track": 1})

    # Convert into VIA bounding box format
    df_pivot["region_shape_attributes"] = df_pivot.apply(
        lambda row: json.dumps(
            {
                "name": "rect",
                "x": row["x"],  # Corrected Top-Left Corner
                "y": row["y"],  # Corrected Top-Left Corner
                "width": row["width"],
                "height": row["height"],
            }
        ),
        axis=1,
    )

    # Keep only required columns
    df_final = df_pivot[
        [
            "filename",
            "file_size",
            "file_attributes",
            "region_count",
            "region_id",
            "region_shape_attributes",
            "region_attributes",
        ]
    ]

    # Save to CSV
    df_final.to_csv(filename, index=False)
    print(f" Saved: {filename}")

In [ ]:
# Convert interpolated dataset and save
convert_to_via_format(ds, "shift.csv")

 Saved: leftshift.csv


If I wouldn't have done this shift, it would give tracking of right corner of the body. 